In [13]:
# Citesc datele pe care urmeaza sa le folosesc pentru submisia finala si pentru care aplic grid search

import pandas as pd
filenames = ['ProiectML\\train_samples.txt','ProiectML\\validation_samples.txt']
with open('ProiectML\\my_train_samples.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)
                
filenames = ['ProiectML\\train_labels.txt','ProiectML\\validation_labels.txt']
with open('ProiectML\\my_train_labels.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)
              
            
df = pd.read_fwf("ProiectML\\my_train_samples.txt", sep='\t', header=None)
df_labels = pd.read_csv("ProiectML\\my_train_labels.txt", sep='\t', header=None)

df_merged = pd.merge(df, df_labels, left_on=0, right_on=0)  #Inner join

# Aplic grid search ca sa gasesc cei mai buni parametrii pentru datele de mai sus

from sklearn.model_selection import GridSearchCV 
from sklearn.pipeline import Pipeline 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

text_clf = Pipeline([('cvect', CountVectorizer()), 
                     ('tfidf_transformer', TfidfTransformer()), 
                     ('cls', MultinomialNB())]) 
parameters = { 'cvect__ngram_range': [(1, 1), (1, 2), (2, 2)], 
              'tfidf_transformer__use_idf': (True, False), 
              'tfidf_transformer__norm': ('l1', 'l2'), 
              'cls__alpha': [1, 1e-1, 1e-2, 1e-3] } 
grid_search= GridSearchCV(text_clf, parameters, cv =10,n_jobs = -1) 
grid_search.fit(df_merged['1_x'].values, df_merged['1_y'].values) 
print(grid_search.best_score_) 
print(grid_search.best_params_)

0.6980696436506312
{'cls__alpha': 0.1, 'cvect__ngram_range': (1, 2), 'tfidf_transformer__norm': 'l2', 'tfidf_transformer__use_idf': False}


0.6980696436506312
{'cls__alpha': 0.1, 'cvect__ngram_range': (1, 2), 'tfidf_transformer__norm': 'l2', 'tfidf_transformer__use_idf': False}

In [14]:
# Verific pe datele de validare

import pandas as pd
df = pd.read_fwf("ProiectML\\train_samples.txt", sep='\t', header=None)
df_labels = pd.read_csv("ProiectML\\train_labels.txt", sep='\t', header=None)

df_merged = pd.merge(df, df_labels, left_on=0, right_on=0) # Inner join intre samples si labels (dupa col 0)

    
# https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
# Tochenizare - construiesc un dictionar de features si transorm doc in vectori de features
from sklearn.feature_extraction.text import CountVectorizer 
cvect = CountVectorizer(ngram_range=(1, 2))   
train_data1 = cvect.fit_transform(df_merged['1_x'].values)
  
# Term frequencies - pana acum aveam aparitia cuv, acum iau frecventa
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer(norm= 'l2', use_idf= False)
train_data = tfidf_transformer.fit_transform(train_data1)

# Definesc si antrenez modelul
from sklearn.naive_bayes import MultinomialNB  
cls = MultinomialNB(alpha = 0.1)
cls.fit(train_data, df_merged['1_y'].values) 

# Citesc datele de vaidare cu acelasi procedeu, fara sa le includ in dictionar
val_data_df = pd.read_fwf("ProiectML\\validation_samples.txt", sep='\t', header=None, encoding = 'utf8')
val_labels_df = pd.read_csv("ProiectML\\validation_labels.txt", sep='\t', header=None)

df_merged_val = pd.merge(val_data_df, val_labels_df, left_on=0, right_on=0)

val_data1 = cvect.transform(val_data_df[1].values)
val_data = tfidf_transformer.transform(val_data1)

predictionn = cls.predict(val_data)

# Afisez matricea de confuzie si raportul
from sklearn import metrics
print(metrics.classification_report(predictionn, df_merged_val['1_y'].values))
print("MATRICEA DE CONFUZIE:")
print(metrics.confusion_matrix(predictionn, df_merged_val['1_y'].values))


              precision    recall  f1-score   support

           0       0.67      0.67      0.67      1305
           1       0.68      0.68      0.68      1351

    accuracy                           0.68      2656
   macro avg       0.68      0.68      0.68      2656
weighted avg       0.68      0.68      0.68      2656

MATRICEA DE CONFUZIE:
[[872 433]
 [429 922]]


              precision    recall  f1-score   support

           0       0.67      0.67      0.67      1305
           1       0.68      0.68      0.68      1351

    accuracy                           0.68      2656
   macro avg       0.68      0.68      0.68      2656
weighted avg       0.68      0.68      0.68      2656

MATRICEA DE CONFUZIE:
[[872 433]
 [429 922]]